<a href="https://colab.research.google.com/github/quangHieu2109/MachineLearing/blob/main/Lab_8_21130356_NgoQuangHieu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The main aim of this lab is to deal with the **pipeline** technique and **MultilayerPerceptron** algorithm

*   **Deadline: 23:59, 06/5/2024**



# Import libraries

In [1]:
# code
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from prettytable import PrettyTable
from sklearn import svm
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn import datasets
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier

from google.colab import drive
drive.mount('/content/gdrive')
%cd '/content/gdrive/MyDrive/ML/dataset'

Mounted at /content/gdrive
/content/gdrive/MyDrive/ML/dataset


#Task 1. With **iris** dataset
*  Apply **pipeline** including preprocessing steps (i.e., **StandardScaler**, **SimpleImputer**, **feature selection**, **KBinsDiscretizer**, …) and classification algorithms (i.e., **Random forest, kNN, Naïve Bayes**).


In [10]:
def _insertTable(t, nameAlg, y_test, y_pred):
  t.add_row([nameAlg, metrics.accuracy_score(y_test, y_pred), metrics.precision_score(y_test, y_pred, average='macro')
  ,metrics.recall_score(y_test, y_pred, average='macro'),metrics.f1_score(y_test, y_pred, average='macro')])
def _trainModel( algos, t):
  for algo in algos:
    pipe_lr = Pipeline(
    [('scl', StandardScaler()),
    ('pca', PCA(n_components=4)),
    (algo.__class__.__name__, algo)]);
    pipe_lr.fit(X_train, y_train)
    y_pred = pipe_lr.predict(X_test)
    _insertTable(t, "pipeline - "+algo.__class__.__name__, y_test, y_pred )
def _comparePerformance(t, algos):
  for algo in algos:
    algo.fit(X_train, y_train)
    y_pred = algo.predict(X_test)
    _insertTable(t, algo.__class__.__name__, y_test, y_pred)



In [ ]:
#code
dataSet = datasets.load_iris();
dataX = dataSet.data
dataY = dataSet.target
table = PrettyTable(['Algorithms', 'Accurance', 'Precision', 'Recall','F1 measure'])
X_train, X_test, y_train, y_test = train_test_split(dataX, dataY, test_size=0.3, shuffle=True)
knn = KNeighborsClassifier();
rf = RandomForestClassifier()
nb = GaussianNB()
algos=[knn, rf, nb]
_trainModel(algos, table)
print(table)




+-----------------------------------+--------------------+--------------------+--------------------+--------------------+
|             Algorithms            |     Accurance      |     Precision      |       Recall       |     F1 measure     |
+-----------------------------------+--------------------+--------------------+--------------------+--------------------+
|  pipeline - KNeighborsClassifier  | 0.9777777777777777 | 0.9777777777777779 | 0.9761904761904763 | 0.9761600681140911 |
| pipeline - RandomForestClassifier | 0.9777777777777777 | 0.9777777777777779 | 0.9761904761904763 | 0.9761600681140911 |
|       pipeline - GaussianNB       | 0.9777777777777777 | 0.9777777777777779 | 0.9761904761904763 | 0.9761600681140911 |
+-----------------------------------+--------------------+--------------------+--------------------+--------------------+


#Task 2. With **fashion** dataset
*   2.1. Apply **MultilayerPerceptron** classification with 1 hidden layer
having 10 nodes

In [3]:
# code
trainSet = pd.read_csv("fashion_train.csv")
testSet = pd.read_csv("fashion_test.csv")

X_train = trainSet.drop('y', axis=1);
y_train= trainSet[['y']]

X_test = testSet.drop('y', axis=1);
y_test= testSet[['y']]
# X_train = StandardScaler().fit_transform(X_train)
# y_train = StandardScaler().fit_transform(y_train)

clf = MLPClassifier(solver='adam', alpha=1e-5,
 hidden_layer_sizes=(10,), random_state=1)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print("Accurancy: ", accuracy_score(y_test, y_pred))



/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1098: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accurancy:  0.165


*   2.2. Apply **MultilayerPerceptron** algorithm with the following settings (the first hidden layer has 250 neuron, the second one has 100 neurons).

In [ ]:
# code
clf = MLPClassifier(solver='adam', alpha=1e-5,
 hidden_layer_sizes=(250,100,), random_state=1)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print("Accurancy: ", accuracy_score(y_test, y_pred))

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1098: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accurancy:  0.757


*   2.3. Find the best hyperparameters using **GridSearchCV**

In [ ]:
#code
MLPCgrid_params = {
 'hidden_layer_sizes': [(150,100,50), (120,80,40), (100,50,30)],
 'max_iter': [50, 100, 150],
 'activation': ['tanh', 'relu'],
 'solver': ['sgd', 'adam'],
 'alpha': [0.0001, 0.05],
 'learning_rate': ['constant','adaptive'],
}
clf = MLPClassifier()
grid_rf_class = GridSearchCV(
  estimator = clf,
  param_grid = MLPCgrid_params,
  scoring = 'accuracy',
  n_jobs =4,
  cv=10,
  refit = True,
  return_train_score = True)
grid_rf_class.fit(X_train, y_train)
y_pred = grid_rf_class.predict(X_test)
print(grid_rf_class.best_params_)


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1098: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


{'activation': 'tanh', 'alpha': 0.05, 'hidden_layer_sizes': (150, 100, 50), 'learning_rate': 'adaptive', 'max_iter': 150, 'solver': 'sgd'}


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (150) reached and the optimization hasn't converged yet.
  warnings.warn(


*   2.4. Compare the **MultilayerPerceptron** using the best hyperparameters in 2.3 and other classification algorithms (i.e., Random forest, kNN, Naïve Bayes)  in termns of accuracy, precision, recall, and F1

In [ ]:
#grid_params
SMVgrid_params = {'C': [0.1, 1, 10, 100, 1000],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf','linear']}
kNNgrid_params = { 'n_neighbors' : [5,7,9,11,13,15],
               'weights' : ['uniform','distance'],
               'metric' : ['minkowski','euclidean','manhattan']}
RFgrid_params = {
    'n_estimators': [25, 50, 100, 150],
    'max_features': ['sqrt', 'log2', None],
    'max_depth': [3, 6, 9],
    'max_leaf_nodes': [3, 6, 9],
}
LRLRgrid_params= {
    'C': [ 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],
    'max_iter': list(range(100,800,100)),
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
}

NBgrid_params = {'var_smoothing': np.logspace(0,-9, num=100)}

In [ ]:
model = GaussianNB()
algos=[model]


In [11]:
svm_l = svm.SVC(C= 10, gamma= 0.1, kernel= 'rbf')
rf = RandomForestClassifier(max_depth= 9, max_features= 'sqrt', max_leaf_nodes= 6, n_estimators= 150)
knn = KNeighborsClassifier(metric='minkowski', n_neighbors=15, weights='distance')
logistic_regression = LogisticRegression(C= 10, max_iter= 100, penalty='l2', solver='liblinear')
model = GaussianNB(var_smoothing= 0.01519911082952933)
clf = MLPClassifier(
    activation='tanh',
    alpha=0.05,
    hidden_layer_sizes=(150, 100, 50),
    learning_rate='adaptive',
    max_iter=150,
    solver='sgd'
)
algos=[clf, rf, knn, model]

In [ ]:
def _findBestHeperParam(t, param_grid, algo):
  grid_rf_class = GridSearchCV(
  estimator = algo,
  param_grid = param_grid,
  scoring = 'accuracy',
  n_jobs =4,
  cv=10,
  refit = True,
  return_train_score = True)
  grid_rf_class.fit(X_train, y_train)
  y_pred = grid_rf_class.predict(X_test)
  print(grid_rf_class.best_params_)
  _insertTable(t, algo.__class__.__name__, y_test, y_pred )




grid_params=[MLPCgrid_params, RFgrid_params, kNNgrid_params, SMVgrid_params]
for i in range(0, len(algos)):
  _findBestHeperParam(table, algos[i], grid_params[i])

{'var_smoothing': 0.01519911082952933}


In [12]:
#code
table = PrettyTable(['Algorithms', 'Accurance', 'Precision', 'Recall','F1 measure'])
_comparePerformance(table, algos)
print(table)

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1098: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (150) reached and the optimization hasn't converged yet.
  warnings.warn(
<ipython-input-10-7f17eec1463c>:15: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  algo.fit(X_train, y_train)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

+------------------------+-----------+--------------------+--------------------+--------------------+
|       Algorithms       | Accurance |     Precision      |       Recall       |     F1 measure     |
+------------------------+-----------+--------------------+--------------------+--------------------+
|     MLPClassifier      |   0.742   | 0.7373345466584647 | 0.7396491287423912 | 0.7318748548892016 |
| RandomForestClassifier |   0.645   | 0.5833625996436448 | 0.6468281972351004 | 0.5824525003206339 |
|  KNeighborsClassifier  |   0.755   | 0.7705245871885532 | 0.7562478835649278 | 0.7518914732379066 |
|       GaussianNB       |   0.675   | 0.6849581549643521 | 0.6746273385329136 | 0.6559984740548461 |
+------------------------+-----------+--------------------+--------------------+--------------------+


#Task 3. With **breast cancer** dataset

*   3.1. Apply **GridSearchCV** to **MultilayperPerceptron** to find the best hyperparameters (the setting of hyperparameters chosen by students)

In [ ]:
svm_l = svm.SVC(C= 10, gamma= 0.1, kernel= 'rbf')
rf = RandomForestClassifier(max_depth= 9, max_features= 'sqrt', max_leaf_nodes= 6, n_estimators= 150)
knn = KNeighborsClassifier(metric='minkowski', n_neighbors=15, weights='distance')
logistic_regression = LogisticRegression(C= 10, max_iter= 100, penalty='l2', solver='liblinear')
model = GaussianNB(var_smoothing= 0.01519911082952933)
clf = MLPClassifier(
    activation='tanh',
    alpha=0.05,
    hidden_layer_sizes=(150, 100, 50),
    learning_rate='adaptive',
    max_iter=150,
    solver='sgd'
)
algos=[clf, rf, knn, model]

In [16]:
from sklearn.datasets import load_breast_cancer

data = load_breast_cancer()
X = data.data  # Ma trận đặc trưng
y = data.target  # Vector nhãn
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, shuffle=True)
table = PrettyTable(['Algorithms', 'Accurance', 'Precision', 'Recall','F1 measure'])
_comparePerformance(table, algos)


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (150) reached and the optimization hasn't converged yet.
  warnings.warn(


*   3.2. Compare the **MultilayerPerceptron** using the best hyperparameters in 3.1) and other classification algorithms (i.e., Random forest, kNN, Naïve Bayes)  in termns of accuracy, precision, recall, and F1

In [17]:
#code
print(table)

+------------------------+--------------------+--------------------+--------------------+--------------------+
|       Algorithms       |     Accurance      |     Precision      |       Recall       |     F1 measure     |
+------------------------+--------------------+--------------------+--------------------+--------------------+
|     MLPClassifier      | 0.8771929824561403 | 0.8838842975206611 | 0.8461251862891208 |  0.85995085995086  |
| RandomForestClassifier | 0.9532163742690059 | 0.9607366771159875 | 0.9380774962742175 | 0.9478181263350626 |
|  KNeighborsClassifier  | 0.9181286549707602 | 0.9250879437160218 | 0.8961997019374068 | 0.9078947368421053 |
|       GaussianNB       | 0.8888888888888888 | 0.9102608695652175 | 0.851564828614009  | 0.8707894213561345 |
+------------------------+--------------------+--------------------+--------------------+--------------------+


#Task 4. With **mobile price classification** dataset


*   4.1. Build your own Neural Network using **MultilayerPerceptron**  



In [19]:
#code
datasetTrain = pd.read_csv("mobile_train.csv")
datasetTest = pd.read_csv("mobile_test.csv")
dataX = datasetTrain.drop("price_range",axis=1)
dataY = datasetTrain.get("price_range")
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, shuffle=True)
table = PrettyTable(['Algorithms', 'Accurance', 'Precision', 'Recall','F1 measure'])
clf = MLPClassifier(

)
_comparePerformance(table, [clf])
print(table)


+---------------+--------------------+--------------------+--------------------+--------------------+
|   Algorithms  |     Accurance      |     Precision      |       Recall       |     F1 measure     |
+---------------+--------------------+--------------------+--------------------+--------------------+
| MLPClassifier | 0.9181286549707602 | 0.9348746081504702 | 0.8981773823191734 | 0.9108047690014903 |
+---------------+--------------------+--------------------+--------------------+--------------------+


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


*   4.2. Apply **GridSearchCV** to **MultilayperPerceptron** to find the best hyperparameters (the setting of hyperparameters chosen by students)

In [20]:
#code
table = PrettyTable(['Algorithms', 'Accurance', 'Precision', 'Recall','F1 measure'])
clf = MLPClassifier(
    activation='tanh',
    alpha=0.05,
    hidden_layer_sizes=(150, 100, 50),
    learning_rate='adaptive',
    max_iter=150,
    solver='sgd'
)
_comparePerformance(table, [clf])
print(table)

+---------------+--------------------+--------------------+--------------------+--------------------+
|   Algorithms  |     Accurance      |     Precision      |       Recall       |     F1 measure     |
+---------------+--------------------+--------------------+--------------------+--------------------+
| MLPClassifier | 0.8830409356725146 | 0.9053921568627451 | 0.8560562571756601 | 0.8706113801452784 |
+---------------+--------------------+--------------------+--------------------+--------------------+


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (150) reached and the optimization hasn't converged yet.
  warnings.warn(


#Finally,
Save a copy in your Github. Remember renaming the notebook.